In [47]:
import os
import sys
import django
import pandas as pd
import numpy as np

cwd = os.getcwd()

BASE_DIR = os.path.abspath(os.path.join(cwd, '..', '..'))  

if BASE_DIR not in sys.path:
    sys.path.append(BASE_DIR)
    print(f"Adicionado {BASE_DIR} ao sys.path")

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "projetob.settings")

django.setup()

from api.models import *
from asgiref.sync import sync_to_async
from authapp.models import User

In [16]:
async def get_marketdata_df():
    qs = await sync_to_async(list)(MarketData.objects.all().values())
    df = pd.DataFrame(qs)
    return df

# Executa e recebe o DataFrame
df_marketdata = await get_marketdata_df()
print(df_marketdata.head())

      id        date  close   high    low   open  volume symbol
0  17344  1927-12-30  17.66  17.66  17.66  17.66       0  ^GSPC
1  17345  1928-01-03  17.76  17.76  17.76  17.76       0  ^GSPC
2  17346  1928-01-04  17.72  17.72  17.72  17.72       0  ^GSPC
3  17347  1928-01-05  17.55  17.55  17.55  17.55       0  ^GSPC
4  17348  1928-01-06  17.66  17.66  17.66  17.66       0  ^GSPC


In [17]:
df_marketdata.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281215 entries, 0 to 281214
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      281215 non-null  int64 
 1   date    281215 non-null  object
 2   close   281215 non-null  object
 3   high    281215 non-null  object
 4   low     281215 non-null  object
 5   open    281215 non-null  object
 6   volume  281215 non-null  int64 
 7   symbol  281215 non-null  object
dtypes: int64(2), object(6)
memory usage: 17.2+ MB


In [18]:
print("Symbols no BD:", df_marketdata['symbol'].unique())

Symbols no BD: ['^GSPC' 'M2SL' '^FVX' 'NFCI' 'SPGI' 'DHR' 'AAPL' 'UNH' 'TEDRATE' 'MSFT'
 '^BVSP' 'BRFS' 'AMZN' 'NVDA' 'ITSA4.SA' 'PETR4.SA' 'VALE3.SA' 'WEGE3.SA'
 'ABEV3.SA' 'DX=F' 'ZC=F' 'CL=F' 'GC=F' 'NG=F' 'ZS=F' 'ITUB4.SA' 'BRL=X'
 'GOOGL' 'Selic_Over' 'Selic_Over_Long' 'CDI' 'Swap_DI_5Y' 'CSAN3.SA'
 'GSG' 'BZ=F' 'B3SA3.SA' 'CDS_Brasil_5Y' 'BBDC4.SA' 'BTGIMABFIRF.SA'
 'CDS_USA_5Y' 'TSLA' 'META' 'BTC-USD' 'NTNS11.SA']


In [19]:
ptr4 = df_marketdata[df_marketdata['symbol'] == 'ABEV3.SA']

In [20]:
ptr4.tail()

,id,date,close,high,low,open,volume,symbol
281074,281618,2025-11-26,13.67,13.81,13.58,13.80,33961700,ABEV3.SA
281103,281648,2025-11-27,13.72,13.77,13.65,13.70,8093600,ABEV3.SA
281119,281669,2025-11-28,13.86,13.96,13.66,13.70,23319400,ABEV3.SA
281157,281777,2025-12-01,14.01,14.02,13.80,13.84,15984500,ABEV3.SA
281187,281815,2025-12-02,14.00,14.35,13.85,14.01,36109000,ABEV3.SA


In [21]:
async def get_predictiondata_df():
    qs = await sync_to_async(list)(Prediction.objects.all().values())
    return pd.DataFrame(qs)

df_prediction = await get_predictiondata_df()
df_prediction.head()

,id,date,results,symbol,prediction
0,65,2025-10-16,"{'history': {'loss': [0.005554144736379385, 0....",AAPL,"[236.6743927001953, 235.437744140625, 235.1471..."
1,66,2025-10-16,"{'history': {'loss': [0.011958727613091469, 0....",ABEV3.SA,"[11.814314842224121, 11.907608032226562, 11.84..."
2,67,2025-10-16,"{'history': {'loss': [0.008539831265807152, 0....",AMZN,"[218.88583374023438, 220.56967163085938, 218.8..."
3,68,2025-10-16,"{'history': {'loss': [0.00551522197201848, 0.0...",B3SA3.SA,"[12.833183288574219, 12.811125755310059, 12.85..."
4,69,2025-10-16,"{'history': {'loss': [0.008469405584037304, 0....",BBDC4.SA,"[17.071815490722656, 17.06732749938965, 17.035..."


In [22]:
metrics = []
for j, row in df_prediction.iterrows():
    print(f'{row['symbol']}, {row['date']}: {row['results']['metrics']}. {row['results']['selected_features']}')
    metrics.append(row['results']['metrics'])


AAPL, 2025-10-16: {'loss': 0.0016586844576522708, 'mae': 0.032063182443380356}. [['MSFT', 0.9844172373811919], ['DX=F', 0.6697946962078916], ['CSAN3.SA', 0.6597146625370014], ['^FVX', 0.653657570842682]]
ABEV3.SA, 2025-10-16: {'loss': 0.000304333952954039, 'mae': 0.012372687458992004}. [['GSG', -0.7134098212123902], ['BBDC4.SA', 0.6522477825272643], ['DX=F', 0.632422341174613], ['^GSPC', 0.467580294454299]]
AMZN, 2025-10-16: {'loss': 0.0016062386566773057, 'mae': 0.03145165741443634}. [['SPGI', 0.9760020289879526], ['BBDC4.SA', 0.7231559284175962], ['CSAN3.SA', 0.7044271477272491], ['DX=F', 0.6451656779646893]]
B3SA3.SA, 2025-10-16: {'loss': 0.0005087834433652461, 'mae': 0.01698801852762699}. [['^BVSP', 0.9019386876012959], ['CSAN3.SA', 0.8109475942163319], ['TEDRATE', -0.4744264686252334], ['DX=F', 0.521851167823445]]
BBDC4.SA, 2025-10-16: {'loss': 0.0004614948411472142, 'mae': 0.015609415248036385}. [['ITUB4.SA', 0.8228485451791256], ['GSG', -0.609100078420222], ['CSAN3.SA', 0.695619

In [23]:
metrics_df = pd.DataFrame(metrics)
metrics_df.describe()

,loss,mae
count,1728.000000,1728.000000
mean,0.001896,0.025928
std,0.004192,0.021150
min,0.000002,0.001024
25%,0.000367,0.014300
50%,0.000643,0.019202
75%,0.001567,0.029364
max,0.038402,0.166613


In [30]:
ALLOWED_SYMBOLS = [
    'VALE3.SA',
    'PETR4.SA',
    'ITUB4.SA',
    'BBDC4.SA',
    'ABEV3.SA',
    'WEGE3.SA', 
    'B3SA3.SA', 
    'ITSA4.SA', 
    'CSAN3.SA', 
    'BRFS',
    'AAPL',
    'NVDA',
    'MSFT',
    'AMZN',
    'GOOGL',
    'META', 
    'TSLA', 
    'UNH', 
    'DHR', 
    'SPGI']


In [31]:
allowed_symbols_last_predictions = df_prediction[df_prediction['symbol'].isin(ALLOWED_SYMBOLS)].sort_values(by='date').groupby('symbol').tail(1)

In [35]:
allowed_symbols_last_predictions['used_features'] = allowed_symbols_last_predictions['results'].apply(lambda x: x['selected_features'])
allowed_symbols_last_predictions['mae'] = allowed_symbols_last_predictions['results'].apply(lambda x: x['metrics']['mae'])
allowed_symbols_last_predictions['mse'] = allowed_symbols_last_predictions['results'].apply(lambda x: x['metrics']['loss'])

In [38]:
allowed_symbols_last_predictions = allowed_symbols_last_predictions.drop(columns=['results', 'id','date'])

In [44]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 2000)

In [46]:
allowed_symbols_last_predictions.to_excel('allowed_symbols_last_predictions.xlsx', index=False)

In [25]:
async def get_portfolios_df():
    qs = await sync_to_async(list)(Portfolio.objects.all().values())
    return pd.DataFrame(qs)

df_portfolios = await get_portfolios_df()
df_portfolios.head()

,id,user_id,name,description,created_at,assets,initial_distribution,current_distribution,initial_balance,current_balance
0,11,3,Carteira maigoes,carteira que vai me falir amem.,2025-10-23 20:21:44.433566+00:00,"[{'symbol': 'SPGI', 'price': 479.74, 'quantity...","{'SPGI': 0.514886, 'GOOGL': 0.271857, 'ITUB4.S...","{'SPGI': 0.514886, 'GOOGL': 0.271857, 'ITUB4.S...",1863.48,1863.48
1,10,1,Portfolio cansado,"portfolio da canseira braba, não aguenta mais.",2025-10-19 14:19:58.814666+00:00,"[{'symbol': 'CSAN3.SA', 'price': 5.87, 'quanti...","{'CSAN3.SA': 0.030402, 'GOOGL': 0.655956, 'ITU...","{'CSAN3.SA': 0.030402, 'GOOGL': 0.655956, 'ITU...",1930.77,1930.77
2,9,1,Carteira de Teste 18/10,Carteira para testar a validade do tracking re...,2025-10-18 22:46:41.912239+00:00,"[{'symbol': 'SPGI', 'price': 150, 'quantity': ...","{'SPGI': 0.246508, 'AAPL': 0.64092, 'DHR': 0.0...","{'SPGI': 0.246508, 'AAPL': 0.64092, 'DHR': 0.0...",12170.00,12170.00
3,8,2,123213,isso.,2025-10-16 22:59:06.836892+00:00,"[{'symbol': 'AAPL', 'price': 150.5, 'quantity'...","{'AAPL': 0.682153, 'GOOGL': 0.317847}","{'AAPL': 0.6652, 'GOOGL': 0.3348}",2206.25,3748.55
4,7,1,Carteira Exótica corr,Carteira para fins do coisa ruim. corr,2025-10-15 20:45:21.482036+00:00,"[{'symbol': 'SPGI', 'price': 150, 'quantity': ...","{'SPGI': 0.252525, 'AAPL': 0.656566, 'DHR': 0....","{'SPGI': 0.4167, 'AAPL': 0.5303, 'DHR': 0.053}",11880.00,23359.20


In [26]:
for i in df_portfolios.assets:
    print(i)

[{'symbol': 'SPGI', 'price': 479.74, 'quantity': 2}, {'symbol': 'GOOGL', 'price': 253.3, 'quantity': 2}, {'symbol': 'ITUB4.SA', 'price': 37.49, 'quantity': 5}, {'symbol': 'ABEV3.SA', 'price': 12.07, 'quantity': 10}, {'symbol': 'CSAN3.SA', 'price': 5.95, 'quantity': 15}]
[{'symbol': 'CSAN3.SA', 'price': 5.87, 'quantity': 10}, {'symbol': 'GOOGL', 'price': 253.3, 'quantity': 5}, {'symbol': 'ITUB4.SA', 'price': 37.49, 'quantity': 5}, {'symbol': 'DHR', 'price': 209.06, 'quantity': 2}]
[{'symbol': 'SPGI', 'price': 150, 'quantity': 20}, {'symbol': 'AAPL', 'price': 156, 'quantity': 50}, {'symbol': 'DHR', 'price': 180, 'quantity': 6}, {'symbol': 'PETR4.SA', 'price': 15, 'quantity': 6}, {'symbol': 'BBDC4.SA', 'price': 8, 'quantity': 25}]
[{'symbol': 'AAPL', 'price': 150.5, 'quantity': 10}, {'symbol': 'GOOGL', 'price': 140.25, 'quantity': 5}]
[{'symbol': 'SPGI', 'price': 150, 'quantity': 20}, {'symbol': 'AAPL', 'price': 156, 'quantity': 50}, {'symbol': 'DHR', 'price': 180, 'quantity': 6}]
[{'symb

In [27]:
async def get_trackings_df():
    qs = await sync_to_async(list)(PortfolioTracking.objects.all().values())
    return pd.DataFrame(qs) 

df_trackings = await get_trackings_df()
df_trackings.head()

,id,portfolio_id,date,balance,distribution
0,1,9,2025-10-18 00:00:00+00:00,12170.00,"{'SPGI': 0.246508, 'AAPL': 0.64092, 'DHR': 0.0..."
1,2,10,2025-10-19 11:19:59.286217+00:00,1930.77,"{'CSAN3.SA': 0.030402, 'GOOGL': 0.655956, 'ITU..."
2,11,10,2025-10-21 20:42:57.127456+00:00,1948.83,"{'CSAN3.SA': 0.030017636981479252, 'GOOGL': 0...."
3,12,9,2025-10-21 20:42:57.146760+00:00,12505.85,"{'SPGI': 0.24211853641857564, 'AAPL': 0.648306..."
4,14,9,2025-10-22 01:15:00.148741+00:00,12505.85,"{'SPGI': 0.24211853641857564, 'AAPL': 0.648306..."


In [28]:
portfolio_10_data =  df_trackings[df_trackings['portfolio_id'] == 10]

In [29]:
portfolio_10_data

,id,portfolio_id,date,balance,distribution
1,2,10,2025-10-19 11:19:59.286217+00:00,1930.77,"{'CSAN3.SA': 0.030402, 'GOOGL': 0.655956, 'ITU..."
2,11,10,2025-10-21 20:42:57.127456+00:00,1948.83,"{'CSAN3.SA': 0.030017636981479252, 'GOOGL': 0...."
8,19,10,2025-10-23 20:02:11.926830+00:00,1951.07,"{'CSAN3.SA': 0.029683845215866905, 'GOOGL': 0...."
11,35,10,2025-10-25 14:56:54.848796+00:00,1984.67,"{'CSAN3.SA': 0.02842334980676942, 'GOOGL': 0.6..."
14,38,10,2025-10-29 00:00:00+00:00,2050.02,"{'CSAN3.SA': 0.02788415631254973, 'GOOGL': 0.6..."
17,41,10,2025-10-30 00:00:00+00:00,2086.79,"{'CSAN3.SA': 0.027347751676823434, 'GOOGL': 0...."
20,44,10,2025-11-04 00:00:00+00:00,2066.25,"{'CSAN3.SA': 0.02684604018931474, 'GOOGL': 0.6..."
23,47,10,2025-11-05 00:00:00+00:00,2100.66,"{'CSAN3.SA': 0.02739086300824333, 'GOOGL': 0.6..."
26,50,10,2025-11-06 00:00:00+00:00,2099.68,"{'CSAN3.SA': 0.02722456188343785, 'GOOGL': 0.6..."
29,53,10,2025-11-07 00:00:00+00:00,2068.85,"{'CSAN3.SA': 0.027857490273455222, 'GOOGL': 0...."


In [48]:
async def get_users_df():
    qs = await sync_to_async(list)(User.objects.all().values())
    return pd.DataFrame(qs) 

df_users = await get_users_df()
df_users.head()

,id,password,last_login,is_superuser,username,first_name,last_name,is_staff,is_active,date_joined,email,name,phone,created_at
0,1,pbkdf2_sha256$1000000$jQ0xUvyUaTCF7zkWjeIScc$b...,None,False,matheus123,,,False,True,2025-10-14 01:02:06.780459+00:00,matheus@ematheus.com,Matheus Oli,+55 11 91234-5678,2025-10-14 01:02:07.780487+00:00
1,2,pbkdf2_sha256$1000000$OD2eaiKfIHxHXJd96Sllji$b...,None,False,eliane12345,,,False,True,2025-10-16 22:51:05.387293+00:00,elianeferra@gmail.com,eliane ferraz,+55 11 91234-5678,2025-10-16 22:51:06.952424+00:00
2,3,pbkdf2_sha256$1000000$A8jhTDgnqyjHSr50pRCNsg$y...,None,False,eusoujabrummmco,,,False,True,2025-10-19 19:10:23.012752+00:00,jabrummmco@gmail.com,Jambrumco Junior,+55 88 7777-7777,2025-10-19 19:10:23.912328+00:00
